In [1]:
import fileIO
import splice_files
import validate_timestamps
import coordinate_utilities
import time_utilities

from numpy import linspace
from math import ceil

In [2]:
# enter name and directory of input file
input_filename1  = 'workingDir/Evening_Run-3.gpx'
input_filename2  = 'workingDir/fixed.gpx'

# select data fields to import
data_flags1 = {
    'gpsflag':  True,
    'timeflag': True,
    'eleflag':  True,
    'hrflag':   True, 
    'cadflag':  True, 
    'pwrflag':  False 
}
data_flags2 = {
    'gpsflag':  True,
    'timeflag': False,
    'eleflag':  True,
    'hrflag':   False, 
    'cadflag':  False, 
    'pwrflag':  False 
}

In [41]:
trkptlist1, meta_data1 = fileIO.import_trackpoints(data_flags1, input_filename1)
trkptlist2, meta_data2 = fileIO.import_trackpoints(data_flags2, input_filename2)
# trkptlist2 = trkptlist2[0:32]

4709 points imported from workingDir/Evening_Run-3.gpx.
42 points imported from workingDir/fixed.gpx.


In [4]:
coordinate_utilities.compute_total_distance(trkptlist2)

1294.632330857942

In [5]:
# create list of the % of the distance between each node
ds = coordinate_utilities.distance_per_segment(trkptlist2)
print(ds) # meters between each trkpt

[0, 40.05412112931128, 8.88013232438261, 6.987608313128178, 26.722690932340655, 76.1968438084338, 76.19685017303617, 9.00275400858043, 10.00754339726273, 10.220881749186274, 36.70085897809486, 38.92438433775824, 9.002758400100863, 12.917848824413333, 76.73700059446374, 76.73699642569377, 6.987664922622082, 45.61095278108336, 8.056125384526176, 76.19685859279464, 76.19684636766145, 32.54858776992454, 54.489914988284234, 13.415084349555906, 2.223898532489277, 9.316931889253283, 4.847733219893406, 1.111949265890977, 50.66489758273937, 64.87665031360292, 85.76111476641671, 8.261894168646503, 1.3850772628238346, 15.04070377091601, 34.809638605022904, 1.111949265890977, 51.61635347191471, 27.90114778306853, 30.10952564608945, 41.61154604411767, 5.969839686921907, 29.22017102960451]


Now define how many points need to be added between each node:

In [33]:
trkptlist2_dense = coordinate_utilities.add_interpolated_points(trkptlist2,data_flags2,400)

Original number of points: 42, new number of points: 401.
Original distance: 1294.632330857942, new distance: 1294.6323308599378.


Interpolated points are successfully added when the distance is preserved but the length has increased.

## Generate timestamps for the missing GPX coordinates
First we need to identify the timestamps where the watch got paused

In [7]:
coordinate_utilities.find_gap(trkptlist1, mode=1,epsilon=0.01)

('2024-01-28T20:42:29Z', '2024-01-28T20:49:50Z')

Before splicing the files, let's fix the timestamps.

In [37]:
ts1 = {'year': 2024, 'month': 1, 'day': 28, 'hour': 20, 'minute': 43, 'second': 9}
ts2 = {'year': 2024, 'month': 1, 'day': 28, 'hour': 20, 'minute': 49, 'second': 49}
gap_timestamps = time_utilities.generate_timepoints(ts1,ts2)
# print(gap_timestamps)
print(len(gap_timestamps))

401


Give the trkptlist2_dense the desired timestamps:

In [38]:
trkptlist2_dense2 = time_utilities.add_non_dummy_times(trkptlist2_dense,gap_timestamps)
validate_timestamps.compare_trkpt(trkptlist2_dense2,verbose=False)

Matching 401 timestamps to 401 GPS points.
Timestamp start: 2024-01-28T20:43:09
Timestamp end:   2024-01-28T20:49:49          

Fail: 0, Success: 399


In [39]:
# one last check:
for trkpt  in trkptlist2_dense:
    if trkpt.hr == None:
       trkpt.hr = 0 

Export the file:

In [40]:
data_flags3 = {
    'gpsflag':  True,
    'timeflag': True,
    'eleflag':  True,
    'hrflag':   True, 
    'cadflag':  False, 
    'pwrflag':  False 
}
output_filename = "outputDir/garbagio.gpx"
fileIO.export_trackpoints(data_flags3, trkptlist2_dense, output_filename)

Exporting file...
File exported to outputDir/garbagio.gpx
